# With Dec=5e-10, and others with Andrew's value, add a tanh on conductivity try to stop negative c(EC)

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp,tanh,sqrt;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import traceback
import imageio.v2 as imageio


In [2]:
import sys  
str_path_0 = os.path.abspath(os.path.join(pybamm.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P2"))
sys.path.append(str_path_1) 
from Fun_P2_Crack_simp import (
    GetScan,
    recursive_scan,
    GetSol_dict,
    Para_init,
    Run_model_wwo_dry_out ,
    Cal_new_con_Update,
    Run_Model_Base_On_Last_Solution,
    Run_Model_Base_On_Last_Solution_RPT,
    write_excel_xlsx,
    Get_initial_cap,
    Get_initial_cap2
)

In [3]:
def electrolyte_conductivity_Nyman2008Exp_wEC(c_e,c_EC, T):
    x = c_EC / c_e
    coff = 1  
    ratio = ( coff/2 + coff/2 *  tanh((x-4.541*0.5)*1.5))
    sigma_e = (
        ratio * (
        0.1 * 0.06248 * (1+298.15-0.05559) * 
        (c_e/1e3) * (1 - 3.084 *sqrt(c_e/1e3) 
        + 1.33 *(1+ 0.03633 *(exp(1000/298.15))*c_e/1e3)   ) 
        / (1+(c_e/1e3)**4*( 0.00795 *exp(1000/298.15))) )
    )
    return sigma_e
def electrolyte_conductivity_Nyman2008Exp(c_e,c_EC, T):
    sigma_e = (
        (
        0.1 * 0.06248 * (1+298.15-0.05559) * 
        (c_e/1e3) * (1 - 3.084 *sqrt(c_e/1e3) 
        + 1.33 *(1+ 0.03633 *(exp(1000/298.15))*c_e/1e3)   ) 
        / (1+(c_e/1e3)**4*( 0.00795 *exp(1000/298.15))) )
    )
    return sigma_e

In [4]:
V_max = 4.2;        V_min = 2.5; D_ec_sei = 1e-19;
total_cycle = 200; save_at_cycles= 20
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 1 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * total_cycle ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 240 s", )    ] * 1 )   

In [5]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R7/'; 
Target  = 'a3_2_Long_Dsol=1e-19_add_tanh_conduct/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [6]:
De = 3e-10;Dec = 5e-10; Dcross  = 3e-11; Xi=-1.4; t_0plus=0.28

In [8]:
Model_d = pybamm.lithium_ion.DFN(     
    options={
        "SEI":"solvent-diffusion limited",          
        "SEI film resistance":"distributed",          
        "SEI porosity change":"true",     
        "solvent diffusion": "EC wo refill"     } )
Model_dre = pybamm.lithium_ion.DFN(     
    options={
        "SEI":"solvent-diffusion limited",          
        "SEI film resistance":"distributed",          
        "SEI porosity change":"true",     
        "solvent diffusion": "EC w refill"     } )
Model_s = pybamm.lithium_ion.DFN(     
    options={
        "SEI":"solvent-diffusion limited",          
        "SEI film resistance":"distributed",          
        "SEI porosity change":"true",     
        "solvent diffusion": "none"     } )
Models = [Model_s,Model_d,Model_dre]
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008Exp";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);
Para_0['EC transference number'] =    Xi
Para_0['Cation transference number'] =     t_0plus
Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = Dcross
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =  Dcross
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =  Dec
Para_0['Ratio of lithium moles to SEI moles'] =  1
Para_0['EC diffusivity in SEI [m2.s-1]'] =  D_ec_sei
Para_0.update({
    'Electrolyte conductivity [S.m-1]':
    electrolyte_conductivity_Nyman2008Exp_wEC}, )
Para_0['Ratio of lithium moles to SEI moles'] =  1
for model in Models:
    c_e = model.variables["Electrolyte concentration [mol.m-3]"]
    T = model.variables["Cell temperature [K]"]
    c_EC = model.variables["EC concentration [mol.m-3]"]
    model.variables["c(EC) over c(Li+)"] = c_EC / c_e
    model.variables["Electrolyte conductivity [S.m-1]"] =(
        Para_0['Electrolyte conductivity [S.m-1]'](c_e,c_EC, T))
    model.variables["Electrolyte diffusivity [m2.s-1]"] =(
        Para_0['Electrolyte diffusivity [m2.s-1]'](c_e,c_EC, T))

Confirm: using solvent-diffusion limited
using EC wo refill for Li+
Confirm: using solvent-diffusion limited
using EC w refill for Li+
Confirm: using solvent-diffusion limited


Xi =  list(np.linspace(-1.8, -1.0, 10))
print(type(Xi))

In [10]:
Sim = [];Sol=[]; 
for model in Models:
    var = pybamm.standard_spatial_vars  
    mesh_thick = 20; mesh_par=20;
    var_pts = {
        var.x_n: int(mesh_thick),  
        var.x_s: int(mesh_thick),  
        var.x_p: int(mesh_thick),  
        var.r_n: int(mesh_par),  
        var.r_p: int(mesh_par),  }  
    sim = pybamm.Simulation(
        model, experiment = Experiment_Long,
        parameter_values = Para_0,
        solver = pybamm.CasadiSolver(),
        #var_pts=var_pts,
        )  
    try:
        sol = sim.solve(save_at_cycles=save_at_cycles,);
        # check whether the solution is finalized 
        print(sol.cycles[total_cycle-1].steps[-1]["Terminal voltage [V]"].entries[-1])
    except:
        print('Fail for model option: ',model.options["solvent diffusion"])
    else:
        Sol.append(sol)   
        print('Succeed for model option: ',model.options["solvent diffusion"])

print(Sol_ddiff["Electrolyte conductivity [S.m-1]"].entries[:,51] )
print(Sol_ddiff["Electrolyte diffusivity [m2.s-1]"].entries[:,-1])

In [ ]:
# post-processing - define dictionary
########################  Output  ########################
keys_loc_RPT = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",

    "CDend Electrolyte diffusivity [m2.s-1]",
    "CDend Electrolyte conductivity [S.m-1]",

    "CDend EC concentration [mol.m-3]",
    "CDend Electrolyte concentration [mol.m-3]",

    "CDend EC flux [mol.m-2.s-1]",
    "CDend EC flux by diffusion [mol.m-2.s-1]",
    "CDend EC flux by migration [mol.m-2.s-1]",
    "CDend EC flux by Li+ [mol.m-2.s-1]",
    "CDend Li+ flux [mol.m-2.s-1]",
    "CDend Li+ flux by diffusion [mol.m-2.s-1]",
    "CDend Li+ flux by migration [mol.m-2.s-1]",
    "CDend Li+ flux by solvent [mol.m-2.s-1]",
]
keys_tim_RPT = [
    # default: CD
    "CD Time [h]",
    "CD Terminal voltage [V]",
]
keys_cyc_RPT = [   # default: CDend
    "Discharge capacity [A.h]",
    "CDend Loss of capacity to SEI [A.h]",
    "CDend Total lithium in electrolyte [mol]",
    "CDend Total EC in electrolyte [mol]",
    "CDend Total EC in electrolyte and SEI [mol]",
]
keys_all_RPT = [keys_loc_RPT,keys_tim_RPT,keys_cyc_RPT];

In [ ]:
cycle_no_long = np.arange(0,total_cycle,save_at_cycles);
#cycle_no_long[0] = 0;
print(cycle_no_long);
print(cycle_no_long[0:-3])


In [ ]:
sol_s = Sol[0]; cycle_no_s = cycle_no_long[0:-6]
step_CD = 0;step_CC=1;step_CV = 2;
my_dict_s = {}; 
for keys in keys_all_RPT:
    for key in keys:
        my_dict_s[key]=[];
for cycle_i in cycle_no_s:
    my_dict_s_old = my_dict_s; del my_dict_s
    my_dict_s = GetSol_dict (my_dict_s_old,keys_all_RPT, sol_s, 
        cycle_i, step_CD , step_CC , -1, step_CV   )


In [ ]:
sol_d = Sol[1]; cycle_no_d = cycle_no_long
my_dict_d = {}; 
for keys in keys_all_RPT:
    for key in keys:
        my_dict_d[key]=[];
for cycle_i in cycle_no_d:
    my_dict_d_old = my_dict_d; del my_dict_d
    my_dict_d = GetSol_dict (my_dict_d_old,keys_all_RPT, sol_d, 
        cycle_i, step_CD , step_CC , -1, step_CV   )


In [ ]:
sol_dre = Sol[2]; cycle_no_dre =  cycle_no_long[0:-5]
my_dict_dre = {}; 
for keys in keys_all_RPT:
    for key in keys:
        my_dict_dre[key]=[];
for cycle_i in cycle_no_dre:
    my_dict_dre_old = my_dict_dre; del my_dict_dre
    my_dict_dre = GetSol_dict (
        my_dict_dre_old,keys_all_RPT, sol_dre , 
        cycle_i, step_CD , step_CC , -1, step_CV   )   


In [ ]:
my_dict_all = [my_dict_s,my_dict_d,my_dict_dre]
cycle_no_all = [cycle_no_s,cycle_no_d,cycle_no_dre]

In [ ]:
fs=17;Num_subplot = 3;
LS=['--','-.','-']; MA = ['o', 's', 'h',] 
fig, axs = plt.subplots(1,Num_subplot, figsize=(18,4.8),tight_layout=True)
for model,my_dict,ls,ma,cycle_no in zip(Models,my_dict_all,LS,MA,cycle_no_all):
    axs[0].plot(
        cycle_no, 
        my_dict["CDend Total lithium in electrolyte [mol]"],marker = ma, 
        linestyle=ls,
        label=model.options["solvent diffusion"] )
    axs[1].plot(
        cycle_no, 
        my_dict["CDend Total EC in electrolyte [mol]"] ,marker = ma, 
        linestyle=ls,
        label=model.options["solvent diffusion"] )
    axs[2].plot(
        cycle_no, 
        my_dict["CDend Total EC in electrolyte and SEI [mol]"] ,marker = ma, 
        linestyle=ls,
        label=model.options["solvent diffusion"] )
for i in range(0,Num_subplot):
    axs[i].set_xlabel("Cycle numbers",   fontdict={'family':'DejaVu Sans','size':fs})
    axs[i].set_ylabel("EC amount [mol]",   fontdict={'family':'DejaVu Sans','size':fs})
    labels = axs[i].get_xticklabels() + axs[i].get_yticklabels(); [label.set_fontname('DejaVu Sans') for label in labels]
    axs[i].tick_params(labelcolor='k', labelsize=fs, width=1) ;  del labels;
    axs[i].legend(prop={'family':'DejaVu Sans','size':fs-2},loc='best',frameon=False)
   
axs[0].set_title("Total Li in electrolyte",   fontdict={'family':'DejaVu Sans','size':fs+1})
axs[1].set_title("Total EC in electrolyte",   fontdict={'family':'DejaVu Sans','size':fs+1})
axs[2].set_title("Total EC in electrolyte and SEI",   fontdict={'family':'DejaVu Sans','size':fs+1})
axs[0].set_ylabel("Li amount [mol]",   fontdict={'family':'DejaVu Sans','size':fs+1})

In [ ]:
fs=17;Num_subplot = 2;
fig, axs = plt.subplots(1,Num_subplot, figsize=(12,4.8),tight_layout=True)
for model,my_dict,cycle_no in zip(Models,my_dict_all,cycle_no_all):
    axs[0].plot(
        cycle_no, 
        my_dict["Discharge capacity [A.h]"],'-o', 
        label=model.options["solvent diffusion"] )
    axs[1].plot(
        cycle_no, 
        my_dict["CDend Loss of capacity to SEI [A.h]"] ,'-o', 
        label=model.options["solvent diffusion"] )
for i in range(0,Num_subplot):
    axs[i].set_xlabel("Cycle numbers",   fontdict={'family':'DejaVu Sans','size':fs})
    axs[i].set_ylabel("Capacity [A.h]",   fontdict={'family':'DejaVu Sans','size':fs})
    labels = axs[i].get_xticklabels() + axs[i].get_yticklabels(); [label.set_fontname('DejaVu Sans') for label in labels]
    axs[i].tick_params(labelcolor='k', labelsize=fs, width=1) ;  del labels;
    axs[i].legend(prop={'family':'DejaVu Sans','size':fs-2},loc='best',frameon=False)
    axs[i].set_title("Discharge capacity",   fontdict={'family':'DejaVu Sans','size':fs+1})
axs[1].set_title("Capacity loss to SEI",   fontdict={'family':'DejaVu Sans','size':fs+1})

In [ ]:
# Plot a pair of loc dependent varibles
def Plot_Loc_Var( key_all, my_dict,colormap ): # for my_dict only
    Num_subplot = len(key_all); # must have 2+ keys
    fig, axs = plt.subplots(1,Num_subplot, figsize=(7*Num_subplot,5),tight_layout=True)
    for i in range(0,Num_subplot):
        cmap_i = mpl.cm.get_cmap(colormap, len(my_dict[ key_all[i]] ) ) 
        if 'Negative' in key_all[i] or 'negative' in key_all[i]:
            x_loc = "x_n [m]";
        elif 'Positive' in key_all[i] or 'positive' in key_all[i]:
            x_loc = "x_p [m]";
        elif 'Seperator' in key_all[i] or 'seperator' in key_all[i]:
            x_loc = "x_s [m]";
        else:
            x_loc = "x [m]";
        for j in range(0,len(my_dict[ key_all[i] ])):
            axs[i].plot(
                my_dict[x_loc], 
                my_dict[ key_all[i] ][j],'-',
                color=cmap_i(j),)
            axs[i].set_title(key_all[i] ,   fontdict={'family':'DejaVu Sans','size':fs-1})
            #axs[1].set_ylabel("Potential [V]",   fontdict={'family':'DejaVu Sans','size':fs})
            axs[i].set_xlabel(x_loc,   fontdict={'family':'DejaVu Sans','size':fs})
            labels = axs[i].get_xticklabels() + axs[i].get_yticklabels(); 
            [label.set_fontname('DejaVu Sans') for label in labels]
            axs[i].tick_params(labelcolor='k', labelsize=fs, width=1) ;  del labels;
            axs[i].ticklabel_format( axis='x', style='sci',scilimits=[-0.01,0.01], useOffset=None, useLocale=None, useMathText=None)
            #axs[i].legend(prop={'family':'DejaVu Sans','size':fs-2},loc='best',frameon=False)  
    return
# need to add judgement for what kind of x to be chosen

In [ ]:
# Plot a pair of loc dependent varibles - within one step
def Plot_Loc_Var_sol( sol,x_loc_all, key_all, cycle, step,colormap  ): # for initial solution object
    Num_subplot = len(key_all); # must have 2+ keys
    fig, axs = plt.subplots(1,Num_subplot, figsize=(7*Num_subplot,5),tight_layout=True)
    for i in range(0,Num_subplot):
        x_loc=x_loc_all[i]; key=key_all[i];
        LinesNmum = len(sol.cycles[cycle].steps[step][key].entries[0,:] )
        cmap_i = mpl.cm.get_cmap(colormap, LinesNmum) 
        for j in range(0,LinesNmum):
            axs[i].plot(
                sol.cycles[cycle].steps[step][x_loc].entries[:,0], 
                sol.cycles[cycle].steps[step][key].entries[:,j], '-',
                color=cmap_i(j),)
            axs[i].set_title(key ,   fontdict={'family':'DejaVu Sans','size':fs-1})
            #axs[1].set_ylabel("Potential [V]",   fontdict={'family':'DejaVu Sans','size':fs})
            axs[i].set_xlabel(x_loc,   fontdict={'family':'DejaVu Sans','size':fs})
            
            labels = axs[i].get_xticklabels() + axs[i].get_yticklabels(); [label.set_fontname('DejaVu Sans') for label in labels]
            
            axs[i].tick_params(labelcolor='k', labelsize=fs, width=1) ;  del labels;
            axs[i].ticklabel_format( axis='x', style='sci',scilimits=[-0.01,0.01], useOffset=None, useLocale=None, useMathText=None)
            #axs[i].legend(prop={'family':'DejaVu Sans','size':fs-2},loc='best',frameon=False)  
    return 